In [15]:
import asyncio
from typing import List, Dict, Optional
import json
import time
from pathlib import Path

import aiohttp
import numpy as np
import pandas as pd
import pendulum
import logging
import requests
from dataclasses import dataclass
from enum import Enum

from sqlalchemy import create_engine, select, text, and_, Column, Text, String, Integer, Float

from sqlalchemy.orm import sessionmaker, declarative_base
from utils.utils import LOCAL_AIRFLOW_PG_URI, LOCAL_PG_URI
from utils.sl_data import SurflineSpots

In [16]:
import nest_asyncio
nest_asyncio.apply()

In [17]:
logging.basicConfig(level=logging.INFO)

In [18]:
Base = declarative_base()

In [19]:
engine = create_engine(LOCAL_PG_URI)
SessionLocal = sessionmaker(bind=engine)

In [20]:
spots = SurflineSpots()

In [21]:
spots_df = spots.process_spots()

INFO:root:30
INFO:root:spots


In [22]:
spots_df.dtypes

spot_id       object
spot_name     object
spot_lon     float64
spot_lat     float64
spot_url      object
dtype: object

In [23]:
spots_df[spots_df["spot_id"] == "5b84820982d0de001a33fa79"]

,spot_id,spot_name,spot_lon,spot_lat,spot_url
1168,5b84820982d0de001a33fa79,Outer Waters,-121.50089,33.000314,https://www.surfline.com/surf-report/outer-wat...


In [24]:
len(spots_df)

1143

In [25]:
len(spots_df['spot_id'])

1143

In [26]:
class SlSpots(Base):
    __tablename__ = 'sl_spots'
    spot_id = Column(Text, primary_key=True)
    spot_name = Column(Text)
    spot_lon = Column(Float)
    spot_lat = Column(Float)
    spot_url = Column(Text) 

In [28]:

def create_tables():
    Base.metadata.create_all(bind=engine)

In [29]:
create_tables()

In [30]:
with SessionLocal() as db:
    stmt = select(SlSpots.spot_id)
    results = db.execute(stmt).scalars().all()

In [35]:
non_dupe_df = spots_df[~spots_df['spot_id'].isin(results)]

In [36]:
non_dupe_df.to_sql('sl_spots', con=engine, if_exists='append', index=False)

0

In [ ]:
def get_sl_spots